# PV Secondary Target Selection

## sga_semimajor_ends
### Method to create the secondary targeting file for the end points on the semi-major axis of large galaxies (from the SGA) in the BGS sample

##### Author: Kelly Douglass (University of Rochester)

See `/project/projectdirs/desi/target/secondary/README` for output data model

### Target classes
1. **End points on the major axis**
2. Points along the major axis
3. Points off-axis

In [1]:
from astropy.table import Table, Column
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord

import numpy as np

import matplotlib.pyplot as plt
%matplotlib notebook

## Parameters

In [2]:
OVERRIDE = True
REF_EPOCH = 2015.5

#output_directory = '/project/projectdirs/desi/target/secondary/indata/'
output_directory = '../'

## Target catalogs

[Siena Galaxy Atlas](https://portal.nersc.gov/project/cosmo/temp/ioannis/SGA-html-dr9alpha/SGA-2arcmin.html)

In [14]:
# Target catalog file names

input_directory = '/Users/kellydouglass/Documents/Research/data/SGA/'

#input_filename = input_directory + 'SGA_BGS_inspecfootprint.fits'
input_filename = input_directory + 'SGA-ellipse-v3.0.kd.fits'

hdu = fits.open(input_filename)
large_galaxies = Table(hdu[1].data)
hdu.close()

In [15]:
large_galaxies[['TYPE', 'SERSIC']][large_galaxies['IN_FOOTPRINT_GRZ']]

TYPE,SERSIC
str6,float32
SER,1.7383994
DEV,4.0
SER,1.5428454
SER,4.436294
SER,4.5138383
SER,4.6850886
SER,1.3623616
SER,1.437142
SER,2.2080953


In [16]:
np.unique(large_galaxies['TYPE'][large_galaxies['IN_FOOTPRINT_GRZ']])

""
DEV
EXP
REX
SER


## Set first priority: points at either end of the major axis

Note: Galaxy centers should already be BGS targets

### Inclination angle minimum

We require a minimum inclination angle of $25^\circ$.

In [17]:
i_min = 25 # degrees

i_boolean = large_galaxies['BA'] <= np.cos(i_min*np.pi/180.)

lg_to_observe = large_galaxies[i_boolean]

### Disk galaxies

We require an exponential profile fit from Tractor.

In [18]:
disk_to_observe = lg_to_observe[lg_to_observe['TYPE'] == 'EXP']

### Within the SGA

We require all galaxies to have a minimum diameter at the 25 magnitude of 20".

In [19]:
#SGA_disks = disk_to_observe[disk_to_observe['inSGA']]
SGA_disks = disk_to_observe

## Calculate (ra, dec) of major axis end points

We set two fiber target locations at either end of the major axis at a distance of $0.4R(26)$, where the value of 0.4 was determined based on the results of the measurements taken during SV.

In [20]:
center_ra = SGA_disks['RA']   # degrees
center_dec = SGA_disks['DEC'] # degrees

centers = SkyCoord(center_ra*u.deg, center_dec*u.deg)

phi = SGA_disks['PA']*u.deg
r26 = 0.5*SGA_disks['DIAM']*u.arcmin

x = 0.4 # Determined during SV

# Maximum distance along the semi-major axis from the center coordinate for our endpoints
delta_a = x*r26

# Target positions
fiber1 = centers.directional_offset_by(phi, x*r26)
fiber2 = centers.directional_offset_by(phi + 180*u.deg, x*r26)

fiber_ra = np.concatenate((fiber1.ra, fiber2.ra))
fiber_dec = np.concatenate((fiber1.dec, fiber2.dec))

### Write target list to file

In [21]:
N_targets = len(fiber_ra)

In [22]:
disk_edges = Table([Column(fiber_ra, name='RA'), 
                    Column(fiber_dec, name='DEC'), 
                    Column(np.zeros(N_targets, dtype='>f4'), name='PMRA'), 
                    Column(np.zeros(N_targets, dtype='>f4'), name='PMDEC'), 
                    Column(REF_EPOCH*np.ones(N_targets, dtype='>f4'), name='REF_EPOCH'),
                    Column(OVERRIDE*np.ones(N_targets, dtype='bool'), name='OVERRIDE')])

In [24]:
disk_edges.write(output_directory + 'sga_semimajor_ends.fits', format='fits')

## Target statistics

In [25]:
print('The number of disk galaxies is', len(SGA_disks))
print('The number of targets is', N_targets)

sky_chunk_boolean = np.logical_and.reduce([disk_edges['RA'] > 150, disk_edges['RA'] < 250, 
                                           disk_edges['DEC'] > 0, disk_edges['DEC'] < 50])

num_targets_in_sky_chunk = np.sum(sky_chunk_boolean)

sky_area = 100*50

print('The number of fiber placements in this portion of the sky is', 
      num_targets_in_sky_chunk)
print('The number of fiber placements per square degree is', 
      num_targets_in_sky_chunk/sky_area)

The number of disk galaxies is 408323
The number of targets is 816646
The number of fiber placements in this portion of the sky is 209642
The number of fiber placements per square degree is 41.9284
